In [ ]:
'''
없으면 설치
!pip install BeautifulSoup4
!pip install selenium
'''

In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from urllib import parse
import time
import pandas as pd
import re
from tqdm import tqdm

In [19]:
#import os -> os.getdir() 해서 나온 디렉토리에 chrome driver.exe 를 넣어주시면 됩니다
driver = webdriver.Chrome()

In [20]:
driver.maximize_window()
driver.get('https://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')
time.sleep(1)

In [ ]:
'''
#년도 설정 (2021년 이전 데이터 크롤링하려면 필요)
#option[1]=2021년, option[2]=2020년, ... (근데 2021년이 디폴트라 따로 설정안해도 됨!)
year_start=driver.find_element(By.XPATH, '//*[@id="ptsRafYearStart"]/option[1]')
year_start.click()
year_end=driver.find_element(By.XPATH, '//*[@id="ptsRafYearEnd"]/option[1]')
year_end.click()
'''

In [5]:
#제주도로 설정
region=driver.find_element(By.XPATH, '//*[@id="ptsRafSido"]/option[17]')
region.click()

In [6]:
#제주도 지역 전체 설정
select=driver.find_element(By.XPATH, '//*[@id="ptsRafSigungu"]/option[1]')
select.click()

In [7]:
#중상, 경상, 부상신고도 체크
acc=driver.find_element(By.XPATH, '//*[@id="ptsRafCh1AccidentContent"]/li[2]/input')
acc.click()
acc=driver.find_element(By.XPATH, '//*[@id="ptsRafCh1AccidentContent"]/li[3]/input')
acc.click()
acc=driver.find_element(By.XPATH, '//*[@id="ptsRafCh1AccidentContent"]/li[4]/input')
acc.click()

In [8]:
#검색
driver.find_element(By.XPATH, '//*[@id="regionAccidentFind"]/div[2]/p/a').send_keys(Keys.ENTER)

In [9]:
#크롤링
#445 대신 검색결과로 나온 건수 개수로 넣어주면 됨
coord_ls = []
srs_ls = []

for i in tqdm(range(0,4030)):
    driver.execute_script(f'gis.srh.msh.selectListUi({i}, 11);') # 개별 사고지점
    time.sleep(1) # 로드
    bbox = driver.find_elements(By.CLASS_NAME,'olTileImage')[-1]
    src = parse.unquote(bbox.get_attribute('src')) # get src and unquote from 16byte
    src_split = src.split('&')
    bbox_text = src_split[-3][5:]
    coords = [float(i) for i in bbox_text.split(',')]
    x, y = (coords[0]+coords[2])/2, (coords[1]+coords[3])/2 # center position
    srs = src_split[-4][4:]
    coord_ls.append([x, y])
    srs_ls.append(srs)

100%|████████████████████████████████████████████████████████████████████████████| 4030/4030 [5:06:34<00:00,  4.56s/it]


In [10]:
#좌표 결과
coord_ls

[[152993.47164216, -829.4181260222849],
 [144945.75293161, -7671.3605867726],
 [188925.98462168, -6416.538675741],
 [133325.75559744, -32015.712942089],
 [129708.22416878, -10427.433519363],
 [152348.42554757, -1279.6719814443],
 [131202.31348211, -10559.297554963],
 [155202.06498378, 2107.1934135076],
 [180201.18234215, -20670.639690566],
 [150581.29507894, -1581.2766447761],
 [168889.8623786, 5402.2781694923],
 [130298.75743122, -29699.471441508],
 [161586.15520646, 3158.3040665838],
 [164606.38338166, -9561.5251804727],
 [173790.81975579, -23580.381322405],
 [131921.92575723, -8143.099797905499],
 [150741.01349799, -1096.3485960683],
 [156906.72034894, 1259.1201102617],
 [159610.65521207, -26453.23993434],
 [181852.93090759, -20701.73820058],
 [152767.01187908, -749.48178984312],
 [153184.36758218, 1282.2005995656],
 [152019.3422555, -453.98341068323],
 [161196.54281555, -25481.322070931],
 [168314.1991014, 5705.6089242556],
 [159586.59470556, -26233.280630545],
 [140462.78117253, -

In [12]:
#좌표 list를 array로 변환
import numpy as np
coord=np.array(coord_ls)

In [13]:
import pyproj
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5181
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fy, fx])[0]

In [14]:
# 좌표계 정보 설정
p1_type = "epsg:5181"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)
result

C:\Users\EunSeon\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\EunSeon\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\EunSeon\AppData\Local\Temp\ipykernel_20840\1981099569.py:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy

array([[ 33.4851382 , 126.49422114],
       [ 33.42307016, 126.40805265],
       [ 33.43573905, 126.88091326],
       ...,
       [ 33.22365383, 126.25529051],
       [ 33.42747093, 126.48993887],
       [ 33.25467305, 126.57136128]])

In [15]:
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

In [16]:
address = geocoding_reverse('33.4851382, 126.49422114')
print(address) 	

세종호텔, 사장1길, 제주시, 제주특별자치도, 63123, 대한민국


In [17]:
#데이터프레임으로 변환
df = pd.DataFrame(result)
df.to_csv('2020_coord.csv', index=False)